In [ ]:
import pandas as pd
import numpy as np
import pickle

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import  make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


df=pd.read_csv('RTA_preprocessed.csv')
df.head()


In [ ]:
x=df.drop('Accident_severity',axis=1)
y=df['Accident_severity']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
ohe=OneHotEncoder()
ohe.fit(x[['Age_band_of_driver','Sex_of_driver','Educational_level','Vehicle_driver_relation','Driving_experience','Type_of_vehicle','Area_accident_occured','Lanes_or_Medians','Types_of_Junction','Road_surface_type','Light_conditions','Weather_conditions','Type_of_collision','Vehicle_movement','Pedestrian_movement','Cause_of_accident']])

In [ ]:
column_trans=make_column_transformer((OneHotEncoder(categories=ohe.categories_),['Age_band_of_driver','Sex_of_driver','Educational_level','Vehicle_driver_relation','Driving_experience','Type_of_vehicle','Area_accident_occured','Lanes_or_Medians','Types_of_Junction','Road_surface_type','Light_conditions','Weather_conditions','Type_of_collision','Vehicle_movement','Pedestrian_movement','Cause_of_accident']),remainder='passthrough')


In [ ]:
rf=RandomForestClassifier(n_estimators=100,random_state=42)
pipe=make_pipeline(column_trans,rf)
pipe.fit(X_train,y_train)
pipe.score(X_test,y_test)
print(classification_report(y_test,pipe.predict(X_test)))
print(accuracy_score(y_test,pipe.predict(X_test)))

In [ ]:
with open ("rf2_model.pkl","wb") as f:
    pickle.dump(pipe,f)